# Imports

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

# Data Read In

In [2]:
df = pd.read_csv('../../DATA/filled_toxicity_df.csv')

df.head()

,mol_id,MolecularWeight,LogP,TPSA,HBDonors,HBAcceptors,RotatableBonds,FractionCSP3,HeavyAtoms,RingCount,...,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,TOX3021,258.324,1.34240,82.28,1.0,5.0,3.0,0.222222,16.0,2.0,...,1,0,0,0,0,1,0,0,0,0
1,TOX3020,204.229,1.29940,49.41,1.0,2.0,2.0,0.272727,15.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,TOX3024,288.475,5.09030,20.23,1.0,1.0,1.0,0.900000,21.0,4.0,...,0,0,0,0,0,0,0,0,0,0
3,TOX3027,276.424,3.75244,32.34,1.0,2.0,7.0,0.588235,20.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,TOX20800,206.027,-0.99220,135.29,5.0,3.0,2.0,1.000000,11.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.set_index('mol_id', inplace=True)

df.head()

,MolecularWeight,LogP,TPSA,HBDonors,HBAcceptors,RotatableBonds,FractionCSP3,HeavyAtoms,RingCount,AromaticProportion,...,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
mol_id,,,,,,,,,,,,,,,,,,,,,
TOX3021,258.324,1.34240,82.28,1.0,5.0,3.0,0.222222,16.0,2.0,0.5625,...,1,0,0,0,0,1,0,0,0,0
TOX3020,204.229,1.29940,49.41,1.0,2.0,2.0,0.272727,15.0,2.0,0.4000,...,0,0,0,0,0,0,0,0,0,0
TOX3024,288.475,5.09030,20.23,1.0,1.0,1.0,0.900000,21.0,4.0,0.0000,...,0,0,0,0,0,0,0,0,0,0
TOX3027,276.424,3.75244,32.34,1.0,2.0,7.0,0.588235,20.0,1.0,0.3000,...,0,0,0,0,0,0,0,0,0,0
TOX20800,206.027,-0.99220,135.29,5.0,3.0,2.0,1.000000,11.0,0.0,0.0000,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.columns

Index(['MolecularWeight', 'LogP', 'TPSA', 'HBDonors', 'HBAcceptors',
       'RotatableBonds', 'FractionCSP3', 'HeavyAtoms', 'RingCount',
       'AromaticProportion', 'LogS_ESOL', 'PositiveCharges', 'NegativeCharges',
       'FormalCharge', 'AromaticRings', 'AromaticHeterocycles',
       'AliphaticRings', 'MolecularComplexity', 'MolarRefractivity',
       'Heteroatoms', 'HalogenCount', 'PhenolicGroups', 'NR-AR', 'NR-AR-LBD',
       'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma',
       'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53'],
      dtype='object')

In [5]:
subset_0 = df[df['SR-HSE'] == 0].sample(n=884, random_state=42)

subset_1 = df[df['SR-HSE'] == 1]

balanced_df = pd.concat([subset_0, subset_1])

features_df = balanced_df[['MolecularWeight', 'LogP', 'TPSA', 'HBDonors', 'HBAcceptors',
       'RotatableBonds', 'FractionCSP3', 'HeavyAtoms', 'RingCount', 'LogS_ESOL',
       'FormalCharge', 'AromaticRings', 'AromaticHeterocycles',
       'AliphaticRings', 'MolecularComplexity', 'MolarRefractivity']]

target_df = balanced_df[['SR-HSE']]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size=0.33, random_state=42)

# Voting Classifier

### with ```LogisticRegression```, ```RandomForestClassifier``` and ```XGBoost```

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(
    features_df, target_df.values.ravel(),
    test_size=0.2, stratify=target_df, random_state=42
)

logreg = LogisticRegression(max_iter=10000, random_state=42)
logreg_params = {
    'C': [0.1, 1, 10]
}
logreg_grid = GridSearchCV(logreg, logreg_params, cv=5, scoring='roc_auc', n_jobs=1)
logreg_grid.fit(X_train, y_train)
best_logreg = logreg_grid.best_estimator_
rf = RandomForestClassifier(random_state=42)
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10]
}
rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='roc_auc', n_jobs=1)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_
xgb = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1]
}
xgb_grid = GridSearchCV(xgb, xgb_params, cv=5, scoring='roc_auc', n_jobs=1)
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_
voting_clf = VotingClassifier(
    estimators=[
        ('lr', best_logreg),
        ('rf', best_rf),
        ('xgb', best_xgb)
    ],
    voting='soft' 
)
voting_clf.fit(X_train, y_train)

for name, model in voting_clf.named_estimators_.items():
    y_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_proba)
    print(f"{name.upper()} AUC: {auc:.4f}")

y_pred_proba = voting_clf.predict_proba(X_val)[:, 1]
ensemble_auc = roc_auc_score(y_val, y_pred_proba)
print(f"Ensemble AUC: {ensemble_auc:.4f}")

LR AUC: 0.7062
RF AUC: 0.6930
XGB AUC: 0.6961
Ensemble AUC: 0.7127


In [8]:
from sklearn.metrics import classification_report

print("\n--- Classification Reports ---\n")

for name, model in voting_clf.named_estimators_.items():
    y_pred = model.predict(X_val)
    print(f"{name.upper()} Classification Report:")
    print(classification_report(y_val, y_pred))
    print('-' * 60)

# Ensemble model
ensemble_pred = voting_clf.predict(X_val)
print("ENSEMBLE Classification Report:")
print(classification_report(y_val, ensemble_pred))


--- Classification Reports ---

LR Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.88      0.78       177
           1       0.52      0.27      0.36        88

    accuracy                           0.68       265
   macro avg       0.61      0.57      0.57       265
weighted avg       0.65      0.68      0.64       265

------------------------------------------------------------
RF Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.84      0.77       177
           1       0.52      0.35      0.42        88

    accuracy                           0.68       265
   macro avg       0.62      0.59      0.60       265
weighted avg       0.65      0.68      0.66       265

------------------------------------------------------------
XGB Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.83      0.78       177
   

# 2: Only XGBoost and RF with Tuning

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report
from imblearn.combine import SMOTEENN

# 1. Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    features_df, target_df.values.ravel(),
    test_size=0.2, stratify=target_df, random_state=42
)

# 2. Apply SMOTEEN on training data only
smoteen = SMOTEENN(random_state=42)
X_train_resampled, y_train_resampled = smoteen.fit_resample(X_train, y_train)

# 4. Random Forest + GridSearchCV
rf = RandomForestClassifier(random_state=42)
rf_params = {'n_estimators': [100, 200], 'max_depth': [None, 10]}
rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='roc_auc', n_jobs=1)
rf_grid.fit(X_train_resampled, y_train_resampled)
best_rf = rf_grid.best_estimator_

# 5. XGBoost + GridSearchCV
xgb = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_params = {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1]}
xgb_grid = GridSearchCV(xgb, xgb_params, cv=5, scoring='roc_auc', n_jobs=1)
xgb_grid.fit(X_train_resampled, y_train_resampled)
best_xgb = xgb_grid.best_estimator_

# 6. Voting Classifier
voting_clf = VotingClassifier(
    estimators=[
        ('rf', best_rf),
        ('xgb', best_xgb)
    ],
    voting='soft'
)
voting_clf.fit(X_train_resampled, y_train_resampled)

# 7. AUC + Classification Report for individual models
for name, model in voting_clf.named_estimators_.items():
    y_proba = model.predict_proba(X_val)[:, 1]
    y_pred = model.predict(X_val)
    auc = roc_auc_score(y_val, y_proba)
    print(f"\n{name.upper()} AUC: {auc:.4f}")
    print(f"{name.upper()} Classification Report:\n{classification_report(y_val, y_pred)}")

# 8. Ensemble AUC + Classification Report
y_pred_proba = voting_clf.predict_proba(X_val)[:, 1]
y_pred_ensemble = voting_clf.predict(X_val)
ensemble_auc = roc_auc_score(y_val, y_pred_proba)
print(f"\nEnsemble AUC: {ensemble_auc:.4f}")
print(f"Ensemble Classification Report:\n{classification_report(y_val, y_pred_ensemble)}")


RF AUC: 0.6676
RF Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.62      0.68       177
           1       0.44      0.59      0.50        88

    accuracy                           0.61       265
   macro avg       0.60      0.61      0.59       265
weighted avg       0.65      0.61      0.62       265


XGB AUC: 0.6620
XGB Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.64      0.70       177
           1       0.45      0.58      0.50        88

    accuracy                           0.62       265
   macro avg       0.60      0.61      0.60       265
weighted avg       0.65      0.62      0.63       265


Ensemble AUC: 0.6628
Ensemble Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.62      0.68       177
           1       0.44      0.59      0.50        88

    accuracy                           0.6

# 3: Additional Models: GB, ET, KNN

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import (
    GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, classification_report

import joblib

# --- Define models and hyperparameters ---
model_configs = {
    'gb': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {'n_estimators': [100], 'learning_rate': [0.1]}
    },
    'et': {
        'model': ExtraTreesClassifier(random_state=42),
        'params': {'n_estimators': [100], 'max_depth': [None]}
    },
    'knn': {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5]}
    }
}

# --- Train models in loop using parallel grid search ---
best_models = {}

for name, config in model_configs.items():
    print(f"{name.upper()} Training started")
    grid = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        cv=5,
        scoring='roc_auc',
        n_jobs=1  # Full parallel grid search
    )
    grid.fit(X_train_resampled, y_train_resampled)
    best_models[name] = grid.best_estimator_
    print(f"{name.upper()} Training ended")

# --- Add existing models ---
best_models['rf'] = best_rf
best_models['xgb'] = best_xgb

# --- Soft Voting Ensemble ---
voting_clf = VotingClassifier(
    estimators=[(name, model) for name, model in best_models.items()],
    voting='soft',
    n_jobs=1  # Parallel prediction across models
)
voting_clf.fit(X_train_resampled, y_train_resampled)

# --- Evaluation loop ---
for name, model in voting_clf.named_estimators_.items():
    try:
        y_proba = model.predict_proba(X_val)[:, 1]
    except AttributeError:
        print(f"{name.upper()} does not support predict_proba. Skipping AUC.")
        y_proba = None

    y_pred = model.predict(X_val)
    if y_proba is not None:
        auc = roc_auc_score(y_val, y_proba)
        print(f"\n{name.upper()} AUC: {auc:.4f}")
    print(f"{name.upper()} Classification Report:\n{classification_report(y_val, y_pred)}")

# --- Ensemble Evaluation ---
y_pred_proba = voting_clf.predict_proba(X_val)[:, 1]
y_pred_ensemble = voting_clf.predict(X_val)
ensemble_auc = roc_auc_score(y_val, y_pred_proba)
print(f"\nEnsemble AUC: {ensemble_auc:.4f}")
print(f"Ensemble Classification Report:\n{classification_report(y_val, y_pred_ensemble)}")

GB Training started
GB Training ended
ET Training started
ET Training ended
KNN Training started
KNN Training ended

GB AUC: 0.6657
GB Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.63      0.68       177
           1       0.42      0.53      0.47        88

    accuracy                           0.60       265
   macro avg       0.58      0.58      0.57       265
weighted avg       0.63      0.60      0.61       265


ET AUC: 0.6713
ET Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.69      0.72       177
           1       0.48      0.57      0.52        88

    accuracy                           0.65       265
   macro avg       0.62      0.63      0.62       265
weighted avg       0.67      0.65      0.66       265


KNN AUC: 0.5256
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.53      0.59    

# 4: Added ANN to the Ensemble

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [12]:
X_train, X_val, y_train, y_val = train_test_split(features_df, target_df, test_size=0.3, stratify=target_df, random_state=42)

In [13]:
smote_enn = SMOTEENN(random_state=42)
X_train_res, y_train_res = smote_enn.fit_resample(X_train, y_train)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_val_scaled = scaler.transform(X_val)

In [16]:
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_res.values.ravel())

X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val.values.ravel())

In [17]:
best_et = best_models['et']

In [19]:
# --- Imports ---
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import classification_report, roc_auc_score

# --- ANN Model ---
class SimpleANN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.out = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)  # raw logits

# --- Skorch ANN Wrapper ---
net_ann = NeuralNetClassifier(
    module=SimpleANN,
    module__input_dim=X_train_scaled.shape[1],
    max_epochs=400,
    lr=0.001,
    batch_size=128,
    optimizer=torch.optim.Adam,
    criterion=nn.BCEWithLogitsLoss,
    train_split=None,
    iterator_train__shuffle=True,
    device='cpu'
)

# --- Train ANN ---
X_train_np = X_train_scaled.astype(np.float32)
y_train_np = y_train_res.values.astype(np.float32).reshape(-1, 1)
net_ann.fit(X_train_np, y_train_np)

# --- Extract the trained PyTorch model ---
trained_model = net_ann.module_

# --- Custom Wrapper (no skorch) ---
class FrozenANN(BaseEstimator, ClassifierMixin):
    def __init__(self, torch_model):
        self.torch_model = torch_model
        self.torch_model.eval()  # set to eval mode

    def fit(self, X, y):
        return self  # Do nothing

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= 0.5).astype(int)

    def predict_proba(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X.astype(np.float32))
            logits = self.torch_model(X_tensor)
            probs = torch.sigmoid(logits).numpy()
            probs = np.hstack([1 - probs, probs])  # shape (N, 2)
        return probs

# --- Wrap ANN ---
wrapped_ann = FrozenANN(torch_model=trained_model)

# --- Final Voting Classifier ---
ensemble = VotingClassifier(
    estimators=[
        ('rf', best_rf),
        ('xgb', best_xgb),
        ('et', best_et),
        ('ann', wrapped_ann)
    ],
    voting='soft',
    n_jobs=1
)

# --- Fit tree models only ---
ensemble.fit(X_train_np, y_train_res.values.ravel())

# --- Evaluate ---
X_val_np = X_val_scaled.astype(np.float32)
y_pred_ens = ensemble.predict(X_val_np)
y_proba_ens = ensemble.predict_proba(X_val_np)[:, 1]
auc = roc_auc_score(y_val, y_proba_ens)

print(f"\n✅ Ensemble AUC: {auc:.4f}")
print("✅ Classification Report:\n", classification_report(y_val, y_pred_ens))


  epoch    train_loss     dur
-------  ------------  ------
      1        0.6680  0.0129
      2        0.6508  0.0127
      3        0.6345  0.0121
      4        0.6191  0.0128
      5        0.6035  0.0121
      6        0.5881  0.0139
      7        0.5739  0.0181
      8        0.5605  0.0241
      9        0.5488  0.0135
     10        0.5372  0.0232
     11        0.5289  0.0175
     12        0.5206  0.0129
     13        0.5136  0.0157
     14        0.5067  0.0134
     15        0.5002  0.0150
     16        0.4941  0.0128
     17        0.4876  0.0138
     18        0.4816  0.0113
     19        0.4751  0.0126
     20        0.4685  0.0154
     21        0.4623  0.0130
     22        0.4561  0.0200
     23        0.4490  0.0151
     24        0.4425  0.0143
     25        0.4360  0.0146
     26        0.4290  0.0208
     27        0.4218  0.0247
     28        0.4149  0.0144
     29        0.4083  0.0156
     30        0.4019  0.0146
     31        0.3950  0.0143
     32   

    212        0.0391  0.0197
    213        0.0388  0.0130
    214        0.0384  0.0161
    215        0.0380  0.0152
    216        0.0377  0.0174
    217        0.0370  0.0125
    218        0.0366  0.0124
    219        0.0366  0.0174
    220        0.0367  0.0167
    221        0.0355  0.0130
    222        0.0358  0.0113
    223        0.0344  0.0150
    224        0.0348  0.0135
    225        0.0342  0.0165
    226        0.0341  0.0116
    227        0.0337  0.0185
    228        0.0329  0.0143
    229        0.0333  0.0210
    230        0.0321  0.0135
    231        0.0324  0.0192
    232        0.0315  0.0148
    233        0.0326  0.0125
    234        0.0332  0.0160
    235        0.0320  0.0115
    236        0.0324  0.0125
    237        0.0317  0.0149
    238        0.0304  0.0124
    239        0.0306  0.0160
    240        0.0289  0.0239
    241        0.0302  0.0136
    242        0.0303  0.0192
    243        0.0291  0.0193
    244        0.0284  0.0188
    245   

In [21]:
export_bundle = {
    'rf': best_rf,
    'xgb': best_xgb,
    'et': best_et,
    'ann_state_dict': trained_model.state_dict(),
    'ann_input_dim': X_train_scaled.shape[1],
}

In [22]:
import joblib 

joblib.dump(export_bundle, '../../Models/SR-HSE/final_voting_classifier.pkl')

['../../Models/SR-HSE/final_voting_classifier.pkl']